To Run you'll need:

pandas, numpy, python-highcharts

All are installable via pip

In [55]:
import json
import csv
import os

import pandas as pd
import numpy as np

from highcharts import Highchart

In [49]:
with open('../data/2016-PresidentialElection.json') as data_file:    
    data = json.load(data_file)

## Converting data to a simple csv so we can make a dataframe without issues

We are sorting everything to make sure we don't mess up the ordering of votes --> candidate

In [50]:
with open('data.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quotechar="'", quoting=csv.QUOTE_MINIMAL)
    writer.writerow(["precinctNumber"] + sorted([x['candidate'] for x in data['precincts'][0]['votes']]))
    
    for row in data['precincts']:
        writer.writerow([row['precinctNumber']] + [x['votes'] for x in sorted(row['votes'], key=lambda y: y['candidate'])])


### Now we have a clean csv, lets make a dataframe 

In [51]:
d = pd.read_csv('data.csv', index_col="precinctNumber")
d.head()

,Atwood/Huber,Castle/Bradley,Clinton/Kaine,DeLaFuenta/Steinberg,Hedges/Bayes,Hoefling/Schulin,Johnson/Weld,Keniston/Taylor,Kennedy/Hart,Kopitke/Sorenson,...,Lyttle/Walsh,Maldonado/Terranova,Maturen/Munoz,McMullin/Johnson,Scott/Barnard,Silva/Silva,Smith/White,Soltysik/Walker,Stein/Baraka,Trump/Pence
precinctNumber,,,,,,,,,,,,,,,,,,,,,
2235235101,1,8,359,0,0,1,54,3,1,2,...,0,1,0,1,0,0,2,0,20,289
2235235102,0,2,98,0,0,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,115
2145235201,0,6,727,0,0,0,82,2,0,0,...,1,0,1,10,0,1,0,0,27,385
2145235202,1,8,649,1,0,0,77,1,0,0,...,1,0,0,17,0,0,0,0,17,340
2145235203,0,5,538,0,0,0,77,0,0,0,...,0,0,0,19,0,0,2,0,23,412


### Execute this cell to only show Trump + Clinton

In [56]:
d = d[['Trump/Pence', 'Clinton/Kaine']].copy()
d.head()

,Trump/Pence,Clinton/Kaine
precinctNumber,,
2235235101,289,359
2235235102,115,98
2145235201,385,727
2145235202,340,649
2145235203,412,538


### Make a bunch of tables!

`gen_pie_chart` takes the precent name and the data and returns the chart object

So then we just iterate over all the rows, and `gen_pie_chart` for each row (after a bit of cleanup)

In [53]:
def gen_pie_chart(name, data):
    ## Change me (colors)
    colors = ["#dc3912", "#3366cc","#ff9900","#109618","#990099","#0099c6","#dd4477",
              "#66aa00","#b82e2e","#316395","#994499","#22aa99","#aaaa11","#6633cc",
              "#e67300","#8b0707","#651067","#329262","#5574a6","#3b3eac","#b77322",
              "#16d620","#b91383","#f4359e","#9c5935","#a9c413","#2a778d","#668d1c",
              "#bea413","#0c5922","#743411"]
    options = {
        'chart': {
            'type': 'pie',
            'options3d': {
                'enabled': True,
                'alpha': 45
            }
        },
        'title': {
            'text': "Precinct: " + name
        },
        'subtitle': {
            'text': 'Votes for each candidate'
        },
        'plotOptions': {
            'pie': {
                'innerSize': 50,
                'depth': 45,
                'dataLabels': {
                    'enabled': False
                },
                'colors': colors,
                'showInLegend': True
            }
        },
    }
    H = Highchart(width=550, height=400)
    H.set_dict_options(options)
    H.add_data_set(data, 'pie', 'Votes')
    return H

def formater(data):
    return [[x[0], int(x[1])] for x in zip(data.keys(), data.values)]

for index, row in d.iterrows():
    ## Force row.name to be a string
    name = str(row.name)
    ## Generate and save chart by it's name
    chart = gen_pie_chart(name, formater(row))
    chart.save_file(name)
    
### While we are at it, lets get the data for all combined
chart = gen_pie_chart("All", formater(d.sum()))
chart.save_file("All")
chart

## Cleanup

`_ = ` is to stop jupyter from printing a list of nones

In [54]:
_ = [os.remove(file) for file in os.listdir() if '.html' in file]
_ = [os.remove(file) for file in os.listdir() if '.csv' in file]